「KvsT2.csv」の「派閥」を予測する  
以下のプログラムをコメントに従って実装してください

読み込むCSVファイル  「KvsT2.csv」

In [19]:
# ライブラリのインポート
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
import pickle

# CSVファイルを読み込む
df = pd.read_csv('KvsT2.csv', encoding='shift-jis')

# 変数dfの先頭5行の中身を表示する
print(df.head())

# 各列の欠損値の件数を表示する
print(df.isnull().sum())

    身長    体重  年代 性別  山間部居住    派閥
0  170  60.0  10  男      1   きのこ
1  172  65.0  20  男      0   きのこ
2  170  60.0  30  女      0  たけのこ
3  170  65.0  40  男      0   きのこ
4  177  65.0  10  女      1  たけのこ
身長       0
体重       2
年代       0
性別       0
山間部居住    0
派閥       0
dtype: int64


In [20]:
# 「派閥」列のデータに偏りが有るかどうか(データごとの出現回数)を確認する
# ※なお、今回はデータに「偏りが有る」と判断したものとする
df['派閥'].value_counts()

たけのこ    42
きのこ     15
Name: 派閥, dtype: int64

In [21]:
# 「派閥」列を基準に、「体重」列の『平均値』を表示する
df.groupby('派閥').mean()['体重']

派閥
きのこ     65.266667
たけのこ    66.275000
Name: 体重, dtype: float64

In [22]:
# 「派閥」列のデータと「性別」列でクロス集計して、「体重」の『中央値』を表示する
# 縦軸を「派閥」列、横軸を「性別」列として中央値を表示する
pd.pivot_table(df, index = '派閥', columns = '性別', values = '体重')

性別,女,男
派閥,,
きのこ,65.333333,65.250000
たけのこ,66.869565,65.470588


In [23]:
# 以下の条件に当てはまる「体重」列を 68 に置き換える
# 「派閥」列 = きのこ
# 「性別」列 = 女
# 「体重」列 = NaN
df.loc[(df['派閥'] == 'きのこ') & (df['性別'] == '女') & (df['体重'] == 'NaN')] = 68

# 上記以外の「体重」列の欠損値は 平均値で置き換える
df = df.fillna(df.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [24]:
# 特徴量と正解データを以下の列とする
# 特徴量　…　「身長」「体重」「年代」「山間部居住」「性別」
#              ※「性別」列はダミー変数化する
# 正解データ　…　「派閥」

# 「性別」列以外の特徴量を変数Xに代入する
X = df[['身長', '体重', '年代', '山間部居住']]

# 「性別」列をダミー変数化し、「男」列のみを変数maleに代入する
male = pd.get_dummies(df['性別'], drop_first = True)
# male = pd.__空欄__

# 変数Xに上記の変数maleを横方向に連結した結果を、変数Xに代入しなおす
X = pd.concat([X, male], axis=1)
# X = pd.__空欄__

# 正解データを変数yに代入する
y = df['派閥']

# 変数X, 変数yを「学習で用いるデータ」と「テストで用いるデータ」に分割する
# * テストデータに使用する割合を『30%』、乱数シードは0とする
# * 「学習で用いるデータ」の特徴量を変数X_trainに代入する
# * 「テストで用いるデータ」の特徴量を変数X_testに代入する
# * 「学習で用いるデータ」の正解データを変数y_trainに代入する
# * 「テストで用いるデータ」の正解データを変数y_testに代入する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [25]:
# 決定木を生成する
# * 深さは5とする
# * 乱数シードは0とする
# * 不均衡データへ対処するオプションを使用する
model = tree.DecisionTreeClassifier(max_depth = 5,random_state = 0, class_weight="balanced")

# 「学習で用いるデータ」を使って学習する
model.fit(X_train, y_train)

# 「学習で用いるデータ」、「テストで用いるデータ」での正解率を表示する
# 「学習で用いるデータ」での正解率を表示する
print('==学習データの正解率==')
print(model.score(X_train, y_train))

# 「テストで用いるデータ」での正解率を表示する
print('==テストデータの正解率==')
print(model.score(X_test, y_test))

==学習データの正解率==
0.9230769230769231
==テストデータの正解率==
0.7777777777777778


In [27]:
# 分岐条件の列、閾値を表示する
# 特徴量(X_train)の列名を表示する
print("==列名==")
print(X_train.columns)

# 決定木の分岐条件の列を表示する
print("==分岐条件の列==")
print(model.tree_.feature)


# 決定木の分岐条件の閾値を表示する
print("==閾値==")
print(model.tree_.threshold)

==列名==
Index(['身長', '体重', '年代', '山間部居住', '男'], dtype='object')
==分岐条件の列==
[ 0  2  4  1 -2  1 -2 -2  1  1 -2 -2 -2  2 -2  0 -2  1 -2 -2 -2]
==閾値==
[173.5  25.    0.5  66.5  -2.   70.   -2.   -2.   69.   65.5  -2.   -2.
  -2.   35.   -2.  166.5  -2.   67.5  -2.   -2.   -2. ]


### 決定木を保存する

In [28]:
# 学習させた決定木を保存する
with open('kouka.pkl', 'wb') as f:
  pickle.dump(model, f)